In [2]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

gcn_msg = fn.copy_src(src='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

In [3]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        
    def forward(self, g, feature):
        with g.local_scope():
            g.ndata['h'] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata['h']
            return self.linear(h)

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GCNLayer(1433, 16)
        self.layer2 = GCNLayer(16, 7)
        
    def forward(self, g, features):
        x = F.relu(self.layer1(g, features))
        x = self.layer2(g, x)
        return x
net = Net()
print(net)

Net(
  (layer1): GCNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=True)
  )
  (layer2): GCNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [22]:
from dgl.data import citation_graph as citegrh
import networkx as nx
def load_cora_data():
    data = citegrh.load_cora()
    features = th.FloatTensor(data.features)
    print("-----------------------------------------------")
    print(features)
    labels = th.LongTensor(data.labels)
    train_mask = th.BoolTensor(data.train_mask)
    test_mask = th.BoolTensor(data.test_mask)
    g = DGLGraph(data.graph)
    return g, features, labels, train_mask, test_mask

In [7]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with th.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)
        correct = th.sum(indices==labels)
        return correct.item()*1.0/len(labels)

In [23]:
import time
import numpy as np
g, features, labels, train_mask, test_mask = load_cora_data()
# Add edges between each node and itself to preserve old node representations
g.add_edges(g.nodes(), g.nodes())
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
dur = []
for epoch in range(50):
    if epoch >=3:
        t0 = time.time()

    net.train()
    logits = net(g, features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    acc = evaluate(net, g, features, labels, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), acc, np.mean(dur)))

Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
-----------------------------------------------
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
Epoch 00000 | Loss 0.0000 | Test Acc 0.7260 | Time(s) nan
Epoch 00001 | Loss 0.0000 | Test Acc 0.7260 | Time(s) nan
Epoch 00002 | Loss 0.0000 | Test Acc 0.7250 | Time(s) nan
Epoch 00003 | Loss 0.0000 | Test Acc 0.7240 | Time(s) 0.0180


F:\Anaconda\lib\site-packages\dgl\data\utils.py:285: UserWarning: Property dataset.feat will be deprecated, please use g.ndata['feat'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
F:\Anaconda\lib\site-packages\dgl\data\utils.py:285: UserWarning: Property dataset.label will be deprecated, please use g.ndata['label'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
F:\Anaconda\lib\site-packages\dgl\data\utils.py:285: UserWarning: Property dataset.train_mask will be deprecated, please use g.ndata['train_mask'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
F:\Anaconda\lib\site-packages\dgl\data\utils.py:285: UserWarning: Property dataset.test_mask will be deprecated, please use g.ndata['test_mask'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
F:\Anaconda\lib\site-packages\dgl\da

Epoch 00004 | Loss 0.0000 | Test Acc 0.7250 | Time(s) 0.0180
Epoch 00005 | Loss 0.0000 | Test Acc 0.7210 | Time(s) 0.0183
Epoch 00006 | Loss 0.0000 | Test Acc 0.7200 | Time(s) 0.0182
Epoch 00007 | Loss 0.0000 | Test Acc 0.7190 | Time(s) 0.0184
Epoch 00008 | Loss 0.0000 | Test Acc 0.7210 | Time(s) 0.0186
Epoch 00009 | Loss 0.0000 | Test Acc 0.7210 | Time(s) 0.0187
Epoch 00010 | Loss 0.0000 | Test Acc 0.7200 | Time(s) 0.0187
Epoch 00011 | Loss 0.0000 | Test Acc 0.7200 | Time(s) 0.0188
Epoch 00012 | Loss 0.0000 | Test Acc 0.7180 | Time(s) 0.0188
Epoch 00013 | Loss 0.0000 | Test Acc 0.7190 | Time(s) 0.0190
Epoch 00014 | Loss 0.0000 | Test Acc 0.7180 | Time(s) 0.0190
Epoch 00015 | Loss 0.0000 | Test Acc 0.7190 | Time(s) 0.0190
Epoch 00016 | Loss 0.0000 | Test Acc 0.7180 | Time(s) 0.0192
Epoch 00017 | Loss 0.0000 | Test Acc 0.7190 | Time(s) 0.0191
Epoch 00018 | Loss 0.0000 | Test Acc 0.7190 | Time(s) 0.0191
Epoch 00019 | Loss 0.0000 | Test Acc 0.7190 | Time(s) 0.0191
Epoch 00020 | Loss 0.000